<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Preprocessing of USPTO (Reaxys formatted)

In [67]:
import pandas as pd
from rdkit import Chem
from rdkit import DataStructs
from rdkit.Chem import AllChem
import numpy as np
from tqdm import tqdm
import pyarrow as pa

In [68]:
"""
Disables RDKit whiny logging.
"""
import rdkit.rdBase as rkrb
import rdkit.RDLogger as rkl
logger = rkl.logger()
logger.setLevel(rkl.ERROR)
rkrb.DisableLog('rdApp.error')


In [69]:
USPTO_elsevier = pd.read_csv('data/USPTO_from_reaxys/uspto_1k_TPL_train_valid.tsv', sep='\t')
USPTO_elsevier.columns

Index(['Unnamed: 0', 'level_0', 'index', 'original_rxn', 'fragments', 'source',
       'year', 'mapped_rxn', 'confidence', 'canonical_rxn_with_fragment_info',
       'canonical_rxn', 'ID', 'reaction_hash', 'reactants', 'products',
       'retro_template', 'template_hash', 'selectivity', 'outcomes',
       'reagents', 'labels'],
      dtype='object')

In [71]:
# create lists for reactants and other stuff
reactant1_list = []
reactant2_list = []

product_list = [] #there's only ever 1 product

reagent1_list = []
reagent2_list = []

other_list = []

for i in range(len(USPTO_elsevier)):
    #handle reactants first
    reactant1 = None
    reactant2 = None
    other = []
    reactants = USPTO_elsevier['reactants'][i].split('.')
    for molecule in reactants:
        if '[' in molecule: #its a reactant
            if not reactant1:
                reactant1 = molecule
            elif not reactant2:
                reactant2 = molecule
            else:
                other +=[molecule]
        else:
            other +=[molecule]

    #then reagents
    reagent1 = None
    reagent2 = None
    reagents = USPTO_elsevier['reagents'][i]
    if reagents == reagents:
        reagents = reagents.split('.')
        for molecule in reagents:
            #populate the first two variables with reagents, add everything else to 'other'
            if 'Pd' in molecule:
                other += [molecule]
            else: 
                if not reagent1:
                    reagent1 = molecule
                elif not reagent2:
                    reagent2 = molecule
                else:
                    other +=[molecule]



            
    reactant1_list += [reactant1]
    reactant2_list += [reactant2]
    product_list += [USPTO_elsevier['products'][i]]
    reagent1_list += [reagent1]
    reagent2_list += [reagent2]
    other_list += [other]


# Read in USPTO data from ORD format (pickled data)

In [1]:
from os import listdir
from os.path import isfile, join
import pandas as pd
from tqdm import tqdm

In [2]:
mypath = 'data/ORD_USPTO/pickled_data/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
#create one big df of all the pickled data
full_df = pd.DataFrame()
for file in tqdm(onlyfiles):
    if file[0] != '.': #We don't want to try to unpickle .DS_Store
        filepath = mypath+file 
        unpickled_df = pd.read_pickle(filepath)
        full_df = pd.concat([full_df, unpickled_df], ignore_index=True)


100%|██████████| 490/490 [05:05<00:00,  1.60it/s]


In [8]:
full_df

,mapped_rxn_0,reactant_0,reactant_1,reactant_2,reactant_3,reactant_4,reactant_5,reagents_0,reagents_1,reagents_2,...,catalyst_4,catalyst_5,catalyst_6,reagents_15,reagents_16,solvent_8,catalyst_7,reactant_9,reagents_17,reagents_18
0,[CH3:1][N:2]([CH2:4][CH:5]1[C:10]([OH:19])([C:...,[OH2:21],[CH3:1][N:2]([CH2:4][CH:5]1[C:10]([OH:19])([C:...,None,None,None,None,,Cl,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[H-].[K+].C1(S)C=CC=CC=1.[CH3:10][N:11]([CH2:1...,[CH3:10][N:11]([CH2:13][C@H:14]1[C@:19]([OH:28...,None,None,None,None,None,[K+],[H-],C(O)COCCO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,[CH3:1][N:2]([CH2:4][C@@H:5]1[C@@:10]([OH:19])...,[CH3:1][N:2]([CH2:4][C@@H:5]1[C@@:10]([OH:19])...,None,None,None,None,None,CN(C[C@H]1[C@](O)(C2C=CC=C(OC)C=2)CCCC1)C,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IC1C(C)=C(I)C(C)=C(I)[C:3]=1[CH3:12].[Mn]([O-]...,IC1C(C)=C(I)C(C)=C(I)[C:3]=1[CH3:12],C([O:22][C:23](=[O:25])[CH3:24])(=O)C,S(=O)(=O)(O)[OH:31],[C:26]([OH:29])(=O)[CH3:27],None,None,,[K+],[Mn]([O-])(=O)(=O)=O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[CH2:1]([I:3])[CH3:2].[Cl:4][C:5]1[CH:10]=[CH:...,[Cl:4][C:5]1[CH:10]=[CH:9][C:8]([C:11]2([CH2:1...,[CH2:1]([I:3])[CH3:2],None,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1771027,[Br:1][C:2]1[CH:7]=[CH:6][C:5]([C:8]2[C:9]3[C:...,[Br:1][C:2]1[CH:7]=[CH:6][C:5]([C:8]2[C:9]3[C:...,C1C(=O)N([Br:29])C(=O)C1,None,None,None,None,CN(C)C=O,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771028,Br[C:2]1[C:3]2[C:8]([CH:9]=[C:10]3[C:15]=1[CH:...,[CH:16]([C:18]1[CH:23]=[CH:22][C:21](B(O)O)=[C...,Br[C:2]1[C:3]2[C:8]([CH:9]=[C:10]3[C:15]=1[CH:...,None,None,None,None,[K+],C1(C)C=CC=CC=1,C1COCC1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771029,[CH:1]([C:3]1[CH:8]=[CH:7][C:6]([C:9]2[C:10]3[...,C1C(=O)N([Br:30])C(=O)C1,[CH:1]([C:3]1[CH:8]=[CH:7][C:6]([C:9]2[C:10]3[...,None,None,None,None,CN(C=O)C,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1771030,[Br:1][C:2]1[C:3]2[C:8]([C:9]([C:16]3[CH:21]=[...,[CH2:24](P(=O)(OCC)OCC)[C:25]1[CH:30]=[CH:29][...,[Br:1][C:2]1[C:3]2[C:8]([C:9]([C:16]3[CH:21]=[...,None,None,None,None,[K+],CC(C)([O-])C,CS(C)=O,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Calculate FP

In [72]:
def calc_fp(lst, radius, nBits):
    ans = []
    for i in tqdm(lst):
        #convert to mole object
        try:
            mol = Chem.MolFromSmiles(i)
            fp = AllChem.GetHashedMorganFingerprint(mol, radius, nBits=nBits)
            array = np.zeros((0, ), dtype=np.int8)
            DataStructs.ConvertToNumpyArray(fp, array)
            ans += [array]
        except:
            ans += [np.zeros((nBits,), dtype=int)]
    return ans

In [73]:
# calculate fingerprints
radius = 3
nBits = 1024

In [74]:
reactant1_list_fp = calc_fp(reactant1_list, radius, nBits)

100%|██████████| 400604/400604 [01:53<00:00, 3534.90it/s]


In [75]:
reactant2_list_fp = calc_fp(reactant2_list, radius, nBits)

100%|██████████| 400604/400604 [02:00<00:00, 3336.46it/s]


In [76]:
product_list_fp = calc_fp(product_list, radius, nBits)

100%|██████████| 400604/400604 [02:36<00:00, 2551.75it/s]


In [77]:
#don't actually need to create a fp of the reagent
reagent1_list_fp = calc_fp(reagent1_list, radius, nBits)


100%|██████████| 400604/400604 [02:03<00:00, 3238.93it/s]


In [78]:
#don't actually need to create a fp of the reagent
reagent2_list_fp = calc_fp(reagent2_list, radius, nBits)

100%|██████████| 400604/400604 [03:08<00:00, 2129.09it/s]


In [79]:
# convert to arrays
p = np.array(product_list_fp)
r1 = np.array(reactant1_list_fp)
r2 = np.array(reactant2_list_fp)
rea1 = np.array(reagent1_list_fp)
rea2 = np.array(reagent1_list_fp)


In [80]:
# calculate rxn difference fp
rxn_diff_fp = p-r1-r2


In [81]:
rxn_diff_fp.shape

(400604, 1024)

In [121]:
# save to paraquet
#X
X_pa_table = pa.table(pd.DataFrame(rxn_diff_fp))
pa.parquet.write_table(X_pa_table, "data/USPTO_from_reaxys/X_rxn_diff_fp.parquet")

AttributeError: module 'pyarrow' has no attribute 'parquet'

# Clustering

In [ ]:
# https://towardsdatascience.com/understanding-k-means-clustering-in-machine-learning-6a6e67336aa1
# let's use sklearn KMeans
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

In [82]:
Kmean = KMeans(n_clusters=10, verbose=1, max_iter=50, random_state=42)

In [83]:
Kmean.fit(rxn_diff_fp)

Initialization complete
Iteration 0, inertia 35800307.0
Iteration 1, inertia 24094394.909923248
Iteration 2, inertia 23335027.908687636
Iteration 3, inertia 23244781.03866532
Iteration 4, inertia 23221065.39939336
Iteration 5, inertia 23200959.972408965
Iteration 6, inertia 23167348.807617933
Iteration 7, inertia 23156281.131787464
Iteration 8, inertia 23137884.43012575
Iteration 9, inertia 23111515.40276076
Iteration 10, inertia 23100266.09307847
Iteration 11, inertia 23097836.11876832
Iteration 12, inertia 23096989.92983185
Iteration 13, inertia 23096350.662857138
Iteration 14, inertia 23093801.901388418
Iteration 15, inertia 23087746.162669748
Iteration 16, inertia 23080321.833504938
Iteration 17, inertia 23073823.66210745
Iteration 18, inertia 23071007.264740333
Iteration 19, inertia 23067090.122566305
Iteration 20, inertia 23062817.405175336
Iteration 21, inertia 23061150.07021009
Iteration 22, inertia 23060678.568692554
Iteration 23, inertia 23060525.49237822
Iteration 24, inerti

KMeans(max_iter=50, n_clusters=10, random_state=42, verbose=1)

In [84]:
Kmean.score(rxn_diff_fp)

-22407942.357398737

In [85]:
Kmean.labels_

array([3, 1, 3, ..., 3, 3, 3], dtype=int32)

# NN modelling

In [112]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [93]:
len(set([1,2,3,4,3]))

4

In [97]:
# create one-hot encoding of reagent1_list

#start by canonicalising all the reagents
reag  = []
for smiles in tqdm(reagent1_list):
    try:
        canon_smiles = Chem.MolToSmiles(Chem.MolFromSmiles(smiles))
        reag += [canon_smiles]
    except TypeError:
        reag += [np.nan]
print(len(set(reag)))
reag = np.array(reag)

100%|██████████| 400604/400604 [01:45<00:00, 3787.27it/s]

5392


In [104]:
# Now do the one-hot encoding
enc = OneHotEncoder(handle_unknown='ignore')
reag_reshaped = reag.reshape(-1, 1)
enc.fit(reag_reshaped)

OneHotEncoder(handle_unknown='ignore')

In [106]:
reag_ohe = enc.transform(reag_reshaped).toarray()

In [108]:
reag_ohe.shape

(400604, 5392)

In [122]:
#y: one hot encoding of reagent 1
y_reag_ohe = pa.table(pd.DataFrame(reag_ohe))
pa.parquet.write_table(y_reag_ohe, "data/USPTO_from_reaxys/reag_ohe.parquet")

AttributeError: module 'pyarrow' has no attribute 'parquet'

## Fully connected

In [113]:
X = rxn_diff_fp
y = reag_ohe
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [109]:
# define the keras model
model = Sequential()
model.add(Dense(6000, input_shape=(1024,), activation='relu'))
model.add(Dense(6000, activation='relu'))
model.add(Dense(5392, activation='sigmoid'))
# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

2022-12-05 02:07:54.934999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [114]:
# fit the keras model on the dataset
model.fit(X_train, y_train, epochs=50, batch_size=10000)


Epoch 1/50
33/33 [==============================] - 760s 23s/step - loss: 0.0039 - accuracy: 0.0724
Epoch 2/50
33/33 [==============================] - 676s 20s/step - loss: 0.0029 - accuracy: 0.1816
Epoch 3/50
33/33 [==============================] - 655s 20s/step - loss: 0.0017 - accuracy: 0.2633
Epoch 4/50
33/33 [==============================] - 623s 19s/step - loss: 0.0013 - accuracy: 0.2789
Epoch 5/50
33/33 [==============================] - 620s 19s/step - loss: 0.0012 - accuracy: 0.2939
Epoch 6/50
33/33 [==============================] - 661s 20s/step - loss: 0.0011 - accuracy: 0.3070
Epoch 7/50
33/33 [==============================] - 651s 20s/step - loss: 0.0010 - accuracy: 0.3202
Epoch 8/50
33/33 [==============================] - 20625s 644s/step - loss: 9.6839e-04 - accuracy: 0.3332
Epoch 9/50
33/33 [==============================] - 673s 20s/step - loss: 9.2571e-04 - accuracy: 0.3442
Epoch 10/50
22/33 [===================>..........] - ETA: 18:49 - loss: 8.9319e-04 - accu

KeyboardInterrupt: 

In [116]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

2504/2504 [==============================] - 118s 47ms/step - loss: 9.1857e-04 - accuracy: 0.3512
Accuracy: 35.12
